In [14]:
# !pip install langchain

In [15]:
## Open AI API Key and Open Source models --Llama, Gemma2, mistral --Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [16]:
# !pip install langchain_groq
# !pip install groq
# !pip install langchain-groq

In [32]:
from langchain_openai import  ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EC5CB40050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EC070CB620>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [33]:
# !pip install langchain_core

In [34]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    # SystemMessage(content="Translate the following from English to French"),
    SystemMessage(content="Translate the following from detected language to French and only give translated results"),
    HumanMessage(content="Hello How are you?")
    
]

result = model.invoke(messages)
result

AIMessage(content='Bonjour, comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 27, 'total_tokens': 38, 'completion_time': 0.02, 'prompt_time': 0.002127945, 'queue_time': 0.018352063000000002, 'total_time': 0.022127945}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-76b3153c-0814-4186-9f56-e9a0750c4f1f-0', usage_metadata={'input_tokens': 27, 'output_tokens': 11, 'total_tokens': 38})

In [35]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

output_parser.invoke(model.invoke(messages))
# output_parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [36]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [37]:
### Using LCEL - Chain the components
chain = model | parser

chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [28]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}: and only give translated results"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}: and only give translated results'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [29]:
result = prompt.invoke({"language": "Hindi", "text":"How are you?"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Hindi: and only give translated results', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})])

In [30]:
result.to_messages()

[SystemMessage(content='Translate the following into Hindi: and only give translated results', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})]

In [31]:
# Chain together components with LCEL
chain = prompt | model | parser 
chain.invoke({"language": "Hindi", "text": "How are you?"})

'आप कैसे हैं? \n'